In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../datasets/cleaned_dataset.csv')
df.sort_values(['Student Number', 'Semester'], inplace=True)

### Adding new columns for extracted features

In [3]:
df.insert(7, 'GPA', 0)
df.insert(8, 'Standing', 'Freshman')
df.insert(9, 'Completed Credits', 0)
df.insert(10, 'Completed ECTS', 0)
df.insert(11, 'GPA Student - Subject', 0)   # avg gpa in courses with that subject
df.insert(12, 'Avg. Grade - Taken', 0)   # avg grade of students who have taken that course so far
df.insert(13, 'Avg. Grade - Students_Subject', 0)   # avg. grade of students in courses with the same subject

### Letter Grade Scale used in Istanbul Sehir University

In [4]:
letter_grade_value = {'A+': 4.1, 'A': 4.0, 'A-': 3.7, 'B+': 3.3, 'B': 3.0, 'B-': 2.7, 'C+': 2.3, 'C': 2.0, 'C-': 1.7, 'D+': 1.3, 'D': 1.0, 'D-': 0.5, 'F': 0.0}

### Calculating GPA, Standing, Completed Credits and ECTS of student in a Time-Series manner

In [5]:
def calculate_gpa(df):
    gpa_dict = {}
    completed_credit = {}
    standing_list = ['Freshman', 'Sophomore', 'Junior', 'Senior']
    for row_idx in df.index:
        student_number = df.iloc[row_idx, 2]
        letter_grade = df.iloc[row_idx, 5]
        semester = df.iloc[row_idx, 14]
        credit = df.iloc[row_idx, 17]
        ects = df.iloc[row_idx, 18]
        completed_credit.setdefault(student_number, {})
        gpa_dict.setdefault(student_number, {})
        gpa_dict[student_number].setdefault(semester, [0, 1, 'Freshman'])   # default list -> [Total, semester of student, standing]
        completed_credit[student_number].setdefault(semester, [0, 0])        
        completed_credit[student_number][semester][0] += credit
        completed_credit[student_number][semester][1] += ects
        gpa_dict[student_number][semester][0] += letter_grade_value[letter_grade] * credit

    for student in completed_credit:
        sorted_semesters = sorted(completed_credit[student])
        for i in range(len(sorted_semesters)):
            if i != 0:
                completed_credit[student][sorted_semesters[i]][0] += completed_credit[student][sorted_semesters[i-1]][0]
                completed_credit[student][sorted_semesters[i]][1] += completed_credit[student][sorted_semesters[i-1]][1]
                gpa_dict[student][sorted_semesters[i]][0] += gpa_dict[student][sorted_semesters[i-1]][0]
    
    for student in gpa_dict:
        sem_counter = 1
        standing_dct = {student: {}}
        standing_counter = 0
        for semester in sorted(gpa_dict[student]):
            year = semester[:4]
            if year not in standing_dct[student] and standing_counter < len(standing_list):
                standing_dct[student][year] = standing_list[standing_counter]
                standing_counter += 1
            gpa_dict[student][semester][1] = sem_counter
            gpa_dict[student][semester][0] /= completed_credit[student][semester][0]
            sem_counter += 1            

        for semester in gpa_dict[student]:
            try:
                gpa_dict[student][semester][2] = standing_dct[student][semester[:4]]
            except KeyError:
                gpa_dict[student][semester][2] = 'Senior'
            
    return gpa_dict, completed_credit

In [6]:
student_gpa, student_credit = calculate_gpa(df)

In [7]:
for row_idx in df.index:
    student_number = df.iloc[row_idx, 2]
    semester = df.iloc[row_idx, 14]
    df.iloc[row_idx, 7] = round(student_gpa[student_number][semester][0], 2)
    df.iloc[row_idx, 8] = student_gpa[student_number][semester][2]
    df.iloc[row_idx, 9] = int(student_credit[student_number][semester][0])
    df.iloc[row_idx, 10] = int(student_credit[student_number][semester][1])

### Calculating 3 distinct features:
* Average GPA of a student with courses with a given subject
* Average grade of students who have taken that course so far
* Average grade of students in courses with the same subject

In [9]:
def avg_grade_student_course(df):
    course_dict = {}
    subject_dict = {}
    student_subject_gpa = {}
    df.sort_values('Semester', inplace=True)
    for idx in df.index:
        student = df.iloc[idx, 2]
        semester = df.iloc[idx, 14].strip()
        course = df.iloc[idx, 0].strip()
        subject = course.split()[0].strip()
        grade = df.iloc[idx, 5].strip()
        course_dict.setdefault(semester, {})
        subject_dict.setdefault(semester, {})
        course_dict[semester].setdefault(course, {})
        subject_dict[semester].setdefault(subject, {})
        course_dict[semester][course].setdefault(grade, 0)
        subject_dict[semester][subject].setdefault(grade, 0)
        subject_dict[semester][subject][grade] += 1
        course_dict[semester][course][grade] += 1
        student_subject_gpa.setdefault(student, {})
        student_subject_gpa[student].setdefault(semester, {})
        student_subject_gpa[student][semester].setdefault(subject, {})
        student_subject_gpa[student][semester][subject].setdefault(grade, 0)
        student_subject_gpa[student][semester][subject][grade] += 1

    for semester in course_dict:
        for course in course_dict[semester]:
            sum_ = 0
            count = 0
            for letter_grade in course_dict[semester][course]:
                sum_ += letter_grade_value[letter_grade] * course_dict[semester][course][letter_grade]
                count += course_dict[semester][course][letter_grade]
            course_dict[semester][course] = (sum_, count)
            
        for subject in subject_dict[semester]:
            sum_ = 0
            count = 0
            for letter_grade in subject_dict[semester][subject]:
                sum_ += letter_grade_value[letter_grade] * subject_dict[semester][subject][letter_grade]
                count += subject_dict[semester][subject][letter_grade]
            subject_dict[semester][subject] = (sum_, count)
            
    for student in student_subject_gpa:
        for semester in student_subject_gpa[student]:
            for subject in student_subject_gpa[student][semester]:
                sum_ = 0
                count = 0
                for letter_grade in student_subject_gpa[student][semester][subject]:
                    sum_ += letter_grade_value[letter_grade] * student_subject_gpa[student][semester][subject][letter_grade]
                    count += student_subject_gpa[student][semester][subject][letter_grade]
                student_subject_gpa[student][semester][subject] = (sum_, count)

    averages = {}
    subject_averages = {}
    for semester in sorted(course_dict):
        for course in course_dict[semester]:
            sem_idx = list(sorted(course_dict)).index(semester)
            averages.setdefault(semester, {})
            cum_sum = 0
            cum_count = 0
            for i in range(sem_idx+1):
                if course in course_dict[list(sorted(course_dict))[i]]:
                    cum_sum += course_dict[list(sorted(course_dict))[i]][course][0]
                    cum_count += course_dict[list(sorted(course_dict))[i]][course][1]
            averages[semester][course] = (cum_sum,cum_count)
        
        for subject in subject_dict[semester]:
            sem_idx = list(sorted(subject_dict)).index(semester)
            subject_averages.setdefault(semester, {})
            cum_sum = 0
            cum_count = 0
            for i in range(sem_idx+1):
                if subject in subject_dict[list(sorted(subject_dict))[i]]:
                    cum_sum += subject_dict[list(sorted(subject_dict))[i]][subject][0]
                    cum_count += subject_dict[list(sorted(subject_dict))[i]][subject][1]
            subject_averages[semester][subject] = (cum_sum,cum_count)    
    
    student_sub_avgs = {}
    for student in student_subject_gpa:
        for semester in sorted(student_subject_gpa[student]):
            for subject in student_subject_gpa[student][semester]:
                student_sub_avgs.setdefault(student, {})
                student_sub_avgs[student].setdefault(semester, {})
                sem_idx = list(sorted(student_subject_gpa[student])).index(semester)
                cum_sum = 0
                cum_count = 0
                for i in range(sem_idx+1):
                    if subject in student_subject_gpa[student][list(sorted(student_subject_gpa[student]))[i]]:
                        cum_sum += student_subject_gpa[student][list(sorted(student_subject_gpa[student]))[i]][subject][0]
                        cum_count += student_subject_gpa[student][list(sorted(student_subject_gpa[student]))[i]][subject][1]
                student_sub_avgs[student][semester][subject] = (cum_sum, cum_count)
    
    for semester in averages:
        for course in averages[semester]:
            sum_ = 0
            count = 0
            tpl = averages[semester][course]
            sum_ += tpl[0]
            count += tpl[1]
            course_dict[semester][course] = sum_ / count
        
        for subject in subject_averages[semester]:
            sum_ = 0
            count = 0
            tpl = subject_averages[semester][subject]
            sum_ += tpl[0]
            count += tpl[1]
            try:
                average_grade = sum_ / count
            except ZeroDivisionError:
                average_grade = 0
            subject_dict[semester][subject] = average_grade
    
    for student in student_sub_avgs:
        for semester in student_sub_avgs[student]:
            for subject in student_sub_avgs[student][semester]:
                sum_ = 0
                count = 0
                tpl = student_sub_avgs[student][semester][subject]
                sum_ += tpl[0]
                count += tpl[1]
                student_subject_gpa[student][semester][subject] = sum_ / count
        
    return course_dict, subject_dict, student_subject_gpa

In [10]:
avg_grade_std, subject_averages, student_sub_gpa = avg_grade_student_course(df)

In [11]:
for row_idx in df.index:
    student = df.iloc[row_idx, 2]
    semester = df.iloc[row_idx, 14]
    course = df.iloc[row_idx, 0].strip()
    subject = course.split()[0].strip()
    df.iloc[row_idx, 11] = student_sub_gpa[student][semester][subject]
    df.iloc[row_idx, 12] = avg_grade_std[semester][course]
    df.iloc[row_idx, 13] = subject_averages[semester][subject]
df.index = range(len(df))
df

,Course Code,Course Title,Student Number,Department Code,Course Level,Letter Grade,Status,GPA,Standing,Completed Credits,Completed ECTS,GPA Student - Subject,Avg. Grade - Taken,Avg. Grade - Students_Subject,Semester,Theoritical,Practical,Course Credit,ECTS,Course Year
0,MGT 511,Küresel Yönetim Düşüncesi,1182,MBA/NT,Graduate,A-,Successful,3.54,Freshman,15,25,3.540000,3.576471,3.324211,2010 - Fall,3,0,3,5,5
1,UNI 123,Textual Analysis and Effective Communication,134,EE,Undergraduate,C-,Successful,3.67,Freshman,18,30,3.666667,2.120833,2.703226,2010 - Fall,3,0,3,5,1
2,UNI 115,Turkish for International Students I,217,CTV,Undergraduate,A-,Successful,3.70,Freshman,3,5,3.700000,3.900000,2.703226,2010 - Fall,3,0,3,5,1
3,MGT 511,Küresel Yönetim Düşüncesi,1227,MBA/NT,Graduate,B,Successful,3.00,Freshman,15,25,3.000000,3.576471,3.324211,2010 - Fall,3,0,3,5,5
4,MGT 541,Uluslararası Pazarlama,1225,MBA/NT,Graduate,B+,Successful,3.33,Freshman,9,15,3.333333,3.238095,3.324211,2010 - Fall,3,0,3,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49342,EECS 216,Signals and Systems,1311,EE,Undergraduate,F,Unsuccessful,2.91,Sophomore,69,115,2.000000,1.005525,2.203085,2014 - Spring,3,0,3,5,2
49343,EECS 202,EECS II: Systems Fundamentals,1311,EE,Undergraduate,D+,Successful,2.91,Sophomore,69,115,2.000000,1.980226,2.203085,2014 - Spring,3,0,3,5,2
49344,EECS 218,Digital Logic Design,1311,EE,Undergraduate,B-,Successful,2.91,Sophomore,69,115,2.000000,2.274528,2.203085,2014 - Spring,3,0,3,5,2
49345,CTV 212,Editing,173,CTV,Undergraduate,C+,Successful,1.97,Sophomore,57,95,2.488889,2.693056,2.924132,2014 - Spring,3,0,3,5,2


In [12]:
df.to_csv('extracted_dataset.csv', index=False)